In [ ]:
###THEANO_FLAGS=mode=FAST_RUN,device=gpu0,floatX=float32 python
import numpy as np
import pandas as pd
import lightgbm as lgb
import scipy.io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from numpy import interp
from google.colab import drive

drive.mount('/content/drive')


def prepare_data(separate=False):
    """
    Load and prepare piRNA-disease association data.
    Returns concatenated feature vectors and labels.
    """
    print("Loading data...")

    # Load similarity matrices and association matrix
    disease_sim = pd.read_csv("/content/drive/My Drive/Data/d2d_do.csv",
                              header=0, index_col=0).to_numpy()
    piRNA_sim = pd.read_csv("/content/drive/My Drive/Data/p2p_smith.csv",
                            header=0, index_col=0).to_numpy()
    associations = pd.read_csv("/content/drive/My Drive/Data/adj.csv",
                               header=0, index_col=0).to_numpy().T

    # Extract relevant portions
    piRNA_sim = piRNA_sim[:2174, :2174]
    associations = associations[:, :2174]

    print(f"Association matrix shape: {associations.shape}")
    print(f"piRNA similarity matrix shape: {piRNA_sim.shape}")
    print(f"Disease similarity matrix shape: {disease_sim.shape}")

    n_diseases = associations.shape[0]
    n_piRNAs = associations.shape[1]

    positive_samples = []
    negative_samples = []
    positive_labels = []
    negative_labels = []

    # Generate feature vectors for each piRNA-disease pair
    for i in range(n_diseases):
        for j in range(n_piRNAs):
            # piRNA feature: similarity scores with all other piRNAs
            piRNA_features = list(piRNA_sim[j, :])

            # Disease feature: similarity scores with all other diseases
            disease_features = list(disease_sim[i, :])

            # Concatenate to create feature vector of size (n_piRNAs + n_diseases)
            feature_vector = piRNA_features + disease_features

            if associations[i, j] == 1:
                # Known association (positive sample)
                positive_samples.append(feature_vector)
                positive_labels.append(1)
            else:
                # Unknown association (potential negative sample)
                negative_samples.append(feature_vector)
                negative_labels.append(0)

    print(f"Number of positive samples: {len(positive_samples)}")
    print(f"Number of negative samples: {len(negative_samples)}")

    # Randomly sample negative samples equal to positive samples
    n_positive = len(positive_samples)
    negative_indices = np.random.permutation(len(negative_samples))[:n_positive]

    selected_negative_samples = [negative_samples[i] for i in negative_indices]
    selected_negative_labels = [negative_labels[i] for i in negative_indices]

    # Combine positive and negative samples
    all_samples = positive_samples + selected_negative_samples
    all_labels = positive_labels + selected_negative_labels

    print(f"Total balanced samples: {len(all_samples)}")
    print(f"Feature vector size: {len(all_samples[0])}")

    return np.array(all_samples), np.array(all_labels)


def build_sparse_autoencoder(input_dim, encoding_dim=128):
    """
    Build a sparse autoencoder for feature extraction.

    Args:
        input_dim: Dimension of input features
        encoding_dim: Dimension of encoded representation

    Returns:
        autoencoder: Full autoencoder model
        encoder: Encoder part only
    """
    # Encoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(350, activation='relu')(input_layer)
    encoded = Dense(250, activation='relu')(encoded)
    encoded = Dense(100, activation='relu')(encoded)

    # Latent representation with L1 regularization for sparsity
    latent = Dense(encoding_dim, activation='relu',
                   activity_regularizer=regularizers.l1(1e-5))(encoded)

    # Decoder
    decoded = Dense(100, activation='relu')(latent)
    decoded = Dense(250, activation='relu')(decoded)
    decoded = Dense(350, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='sigmoid')(decoded)

    # Models
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    encoder = Model(inputs=input_layer, outputs=latent)

    autoencoder.compile(optimizer='adam', loss='mse')

    return autoencoder, encoder


def calculate_performance(y_true, y_pred):
    """Calculate performance metrics."""
    tp = fp = tn = fn = 0

    for true_label, pred_label in zip(y_true, y_pred):
        if true_label == 1:
            if pred_label == 1:
                tp += 1
            else:
                fn += 1
        else:
            if pred_label == 0:
                tn += 1
            else:
                fp += 1

    test_num = len(y_true)
    acc = (tp + tn) / test_num

    if tp == 0 and fp == 0:
        precision = 0
        mcc = 0
        f1_score = 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else:
        precision = tp / (tp + fp)
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)

        denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
        mcc = (tp * tn - fp * fn) / denominator if denominator > 0 else 0

        f1_score = (2 * tp) / (2 * tp + fp + fn)

    return acc, precision, sensitivity, specificity, mcc, f1_score


def piR_LGBM():
    """
    Main function implementing the piR-LGBM model for piRNA-disease association prediction.
    """
    # Load and prepare data
    X, y = prepare_data()

    # Shuffle data
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]

    # Cross-validation setup
    num_folds = 5
    fold_size = len(y) // num_folds

    # Storage for results
    all_performance = []
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)

    print("\n" + "="*60)
    print("Starting 5-Fold Cross-Validation")
    print("="*60)

    for fold in range(num_folds):
        print(f"\n--- Fold {fold + 1}/{num_folds} ---")

        # Split data for this fold
        test_indices = list(range(fold * fold_size, (fold + 1) * fold_size))
        train_indices = list(set(range(len(y))) - set(test_indices))

        X_train = X[train_indices]
        X_test = X[test_indices]
        y_train = y[train_indices]
        y_test = y[test_indices]

        print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")

        # Step 1: Train Sparse Autoencoder
        print("Training Sparse Autoencoder...")
        input_dim = X_train.shape[1]
        autoencoder, encoder = build_sparse_autoencoder(input_dim, encoding_dim=128)

        autoencoder.fit(X_train, X_train,
                       epochs=20,
                       batch_size=100,
                       shuffle=True,
                       verbose=0)

        # Step 2: Extract encoded features
        print("Extracting encoded features...")
        X_train_encoded = encoder.predict(X_train, verbose=0)
        X_test_encoded = encoder.predict(X_test, verbose=0)

        print(f"Encoded feature dimension: {X_train_encoded.shape[1]}")

        # Step 3: Train LightGBM Classifier
        print("Training LightGBM Classifier...")
        params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'verbose': -1
        }

        train_data = lgb.Dataset(X_train_encoded, label=y_train)
        test_data = lgb.Dataset(X_test_encoded, label=y_test, reference=train_data)

        bst = lgb.train(params, train_data, num_boost_round=100,
                       valid_sets=[test_data])

        # Step 4: Predict and evaluate
        y_pred_prob = bst.predict(X_test_encoded)
        y_pred = (y_pred_prob >= 0.5).astype(int)

        # Calculate metrics
        acc, precision, sensitivity, specificity, mcc, f1 = calculate_performance(y_test, y_pred)

        # ROC curve
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        roc_auc = auc(fpr, tpr)

        # Precision-Recall curve
        prec_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        aupr = auc(recall_vals, prec_vals)

        print(f"Accuracy: {acc:.4f}, Precision: {precision:.4f}, Sensitivity: {sensitivity:.4f}")
        print(f"Specificity: {specificity:.4f}, MCC: {mcc:.4f}, AUC: {roc_auc:.4f}, AUPR: {aupr:.4f}")

        all_performance.append([acc, precision, sensitivity, specificity, mcc, roc_auc, aupr, f1])

        # Plot ROC curve for this fold
        plt.plot(fpr, tpr, alpha=0.5,
                label=f'Fold {fold + 1} (AUC = {roc_auc:.4f})')

        # Interpolate for mean ROC curve
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0

    # Calculate mean performance
    mean_tpr /= num_folds
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    mean_performance = np.mean(all_performance, axis=0)
    std_performance = np.std(all_performance, axis=0)

    # Print results
    print("\n" + "="*60)
    print("piR-LGBM Results (5-Fold Cross-Validation)")
    print("="*60)
    print(f"Accuracy:     {mean_performance[0]:.4f} ± {std_performance[0]:.4f}")
    print(f"Precision:    {mean_performance[1]:.4f} ± {std_performance[1]:.4f}")
    print(f"Sensitivity:  {mean_performance[2]:.4f} ± {std_performance[2]:.4f}")
    print(f"Specificity:  {mean_performance[3]:.4f} ± {std_performance[3]:.4f}")
    print(f"MCC:          {mean_performance[4]:.4f} ± {std_performance[4]:.4f}")
    print(f"AUC:          {mean_performance[5]:.4f} ± {std_performance[5]:.4f}")
    print(f"AUPR:         {mean_performance[6]:.4f} ± {std_performance[6]:.4f}")
    print(f"F1-Score:     {mean_performance[7]:.4f} ± {std_performance[7]:.4f}")
    print("="*60)

    # Plot mean ROC curve
    plt.plot(mean_fpr, mean_tpr, 'b--', linewidth=2.5,
            label=f'Mean ROC (AUC = {mean_performance[5]:.4f})')

    plt.xlabel('False Positive Rate (1-Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.title('ROC Curve: piR-LGBM (5-Fold Cross-Validation)')
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Save results
    scipy.io.savemat('piR_LGBM_results.mat', {
        'mean_fpr': mean_fpr,
        'mean_tpr': mean_tpr,
        'mean_auc': mean_performance[5],
        'all_performance': all_performance
    })


if __name__ == "__main__":
    piR_LGBM()